## Proceso ETL

In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Auto Sales data.csv", 
    parse_dates=['ORDERDATE'], 
    dayfirst=True)

In [3]:
#Creacion de la nueva columna DAYS_SINCE_LASTORDER basado en la diferencia de tiempo
#Primero elimino la columna existente

data = data.drop('DAYS_SINCE_LASTORDER', axis=1)

#Basado en la diferencia de tiempo creo la nueva columna DAYS_SINCE_LASTORDER

from datetime import datetime
#Especifico la fecha
temp_date = datetime(2020,6,1)
data['DAYS_SINCE_LASTORDER'] = (temp_date - data['ORDERDATE']).dt.days

In [4]:
data.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,CITY,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,DAYS_SINCE_LASTORDER
0,10107,30,95.70,2,2871.00,2018-02-24,Shipped,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,897 Long Airport Avenue,NYC,10022,USA,Yu,Kwai,Small,828
1,10121,34,81.35,5,2765.90,2018-05-07,Shipped,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,59 rue de l'Abbaye,Reims,51100,France,Henriot,Paul,Small,756
2,10134,41,94.74,2,3884.34,2018-07-01,Shipped,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,75508,France,Da Cunha,Daniel,Medium,701
3,10145,45,83.26,6,3746.70,2018-08-25,Shipped,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,Pasadena,90003,USA,Young,Julie,Medium,646
4,10168,36,96.66,1,3479.76,2018-10-28,Shipped,Motorcycles,95,S10_1678,Technics Stores Inc.,6505556809,9408 Furth Circle,Burlingame,94217,USA,Hirano,Juri,Medium,582


# Separacion de archivos

## Tabla pedidos (Orders)

## Tabla detalles orden (Orders_details)

## Tabla productos (Products)

## Tabla Clientes (Clients)

## Tabla Tamaño del trato (deal_size)

In [11]:
#VERIFICO PRIMERO QUE NO HAYA VALORES NULOS NI DUPLICADOS 

data.isnull().sum()



ORDERNUMBER             0
QUANTITYORDERED         0
PRICEEACH               0
ORDERLINENUMBER         0
SALES                   0
ORDERDATE               0
STATUS                  0
PRODUCTLINE             0
MSRP                    0
PRODUCTCODE             0
CUSTOMERNAME            0
PHONE                   0
ADDRESSLINE1            0
CITY                    0
POSTALCODE              0
COUNTRY                 0
CONTACTLASTNAME         0
CONTACTFIRSTNAME        0
DEALSIZE                0
DAYS_SINCE_LASTORDER    0
dtype: int64

In [13]:
data.duplicated().sum()

np.int64(0)

In [14]:
Orders = data[['ORDERNUMBER','ORDERDATE','STATUS']]

Order_Details = data[['ORDERNUMBER','ORDERLINENUMBER','PRODUCTCODE','QUANTITYORDERED','PRICEEACH','SALES']]

Products = data[['PRODUCTCODE','PRODUCTLINE','MSRP']]

Customers = data[['CUSTOMERNAME','PHONE','ADDRESSLINE1','CITY','POSTALCODE','COUNTRY','CONTACTLASTNAME']]

Deal_Size = data[['DEALSIZE']]

In [16]:
Orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ORDERNUMBER  2747 non-null   int64         
 1   ORDERDATE    2747 non-null   datetime64[ns]
 2   STATUS       2747 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 64.5+ KB


In [17]:
Order_Details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ORDERNUMBER      2747 non-null   int64  
 1   ORDERLINENUMBER  2747 non-null   int64  
 2   PRODUCTCODE      2747 non-null   object 
 3   QUANTITYORDERED  2747 non-null   int64  
 4   PRICEEACH        2747 non-null   float64
 5   SALES            2747 non-null   float64
dtypes: float64(2), int64(3), object(1)
memory usage: 128.9+ KB


In [18]:
Products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PRODUCTCODE  2747 non-null   object
 1   PRODUCTLINE  2747 non-null   object
 2   MSRP         2747 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 64.5+ KB


In [19]:
Customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CUSTOMERNAME     2747 non-null   object
 1   PHONE            2747 non-null   object
 2   ADDRESSLINE1     2747 non-null   object
 3   CITY             2747 non-null   object
 4   POSTALCODE       2747 non-null   object
 5   COUNTRY          2747 non-null   object
 6   CONTACTLASTNAME  2747 non-null   object
dtypes: object(7)
memory usage: 150.4+ KB


In [20]:
Deal_Size.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DEALSIZE  2747 non-null   object
dtypes: object(1)
memory usage: 21.6+ KB


In [25]:
#Creamos index para primarykey valores incrementales desde 1
Orders['ORDER_ID'] = range(1, len(Orders) + 1)

Orders.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_11664\2070477106.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Orders['ORDER_ID'] = range(1, len(Orders) + 1)


,ORDERNUMBER,ORDERDATE,STATUS,ORDER_ID
0,10107,2018-02-24,Shipped,1
1,10121,2018-05-07,Shipped,2
2,10134,2018-07-01,Shipped,3
3,10145,2018-08-25,Shipped,4
4,10168,2018-10-28,Shipped,5


In [27]:
#Creamos index para primarykey valores incrementales desde 1
Order_Details['DETAILS_ID'] = range(1, len(Orders) + 1)

Order_Details.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_11664\1817623533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Order_Details['DETAILS_ID'] = range(1, len(Orders) + 1)


,ORDERNUMBER,ORDERLINENUMBER,PRODUCTCODE,QUANTITYORDERED,PRICEEACH,SALES,DETAILS_ID
0,10107,2,S10_1678,30,95.70,2871.00,1
1,10121,5,S10_1678,34,81.35,2765.90,2
2,10134,2,S10_1678,41,94.74,3884.34,3
3,10145,6,S10_1678,45,83.26,3746.70,4
4,10168,1,S10_1678,36,96.66,3479.76,5


In [28]:
#Guardamos los dataframe en su respectivo arhivo csv para la importar a SQL 

Orders.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Orders.csv", index=False)

Order_Details.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Orders_Details.csv", index=False)

Products.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Products.csv", index=False)

Customers.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Customers.csv", index=False)

Deal_Size.to_csv(r"C:\Users\Admin\Documents\Cars Sales\Archivo csv\Deal_Size.csv", index=False)

In [23]:
# Verifica si hay duplicados en la columna 'ORDERNUMBER'
duplicados = data[data.duplicated(subset='ORDERNUMBER', keep=False)]

# Muestra los duplicados
print("Filas duplicadas en la columna 'ORDERNUMBER':")
print(duplicados)


Filas duplicadas en la columna 'ORDERNUMBER':
      ORDERNUMBER  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER    SALES  \
0           10107               30      95.70                2  2871.00   
1           10121               34      81.35                5  2765.90   
2           10134               41      94.74                2  3884.34   
3           10145               45      83.26                6  3746.70   
4           10168               36      96.66                1  3479.76   
...           ...              ...        ...              ...      ...   
2742        10350               20     112.22               15  2244.40   
2743        10373               29     137.19                1  3978.51   
2744        10386               43     125.99                4  5417.57   
2745        10397               34      62.24                1  2116.16   
2746        10414               47      65.52                9  3079.44   

      ORDERDATE    STATUS  PRODUCTLINE  MSRP PRODUCTC

In [24]:
print("Cantidad de valores duplicados en 'ORDERNUMBER':", data['ORDERNUMBER'].duplicated().sum())


Cantidad de valores duplicados en 'ORDERNUMBER': 2449
